In [1]:
!pip install language_tool_python

In [2]:
import os
import pandas as pd
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import language_tool_python
import requests
import json
from IPython.display import display, HTML

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
model_name="camembert-base"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModel.from_pretrained(model_name)

In [ ]:
def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch

def text_chunking(text, max_chunk_size, model, tokenizer, min_length, min_similarity=0.85):
    """
    Perform variational text chunking on input text.
    
    Args:
        text (str): The input text to chunk.
        max_chunk_size (int): Maximum size of each chunk (in characters).
        min_length (int): Minimum length of each chunk (in characters).
        min_similarity (float): Minimum cosine similarity for chunk merging.
        model: Pretrained language model.
        tokenizer: Pretrained tokenizer.
    
    Returns:
        list: List of text chunks.
    """
    
    
    # Split text into sentences
    sentences = text.split('.')
    
    # Generate sentence embeddings for all sentences
    embeddings = [get_embedding(sentence) for sentence in sentences if sentence.strip()]
    
    chunks = []
    current_chunk = []
    current_embedding = None
    
    for idx, sentence in enumerate(sentences):
        if not sentence.strip():
            continue
        current_chunk.append(sentence)
        chunk_text = ' '.join(current_chunk)
        
        # Check chunk size and the min_length constraint
        if len(chunk_text) > max_chunk_size:
            if len(' '.join(current_chunk[:-1])) >= min_length:
                chunks.append(' '.join(current_chunk[:-1]))
            current_chunk = [sentence]
            current_embedding = get_embedding(sentence)
        else:
            sentence_embedding = embeddings[idx]
            # Compute similarity for merging
            if current_embedding is not None:
                similarity = cosine_similarity([current_embedding], [sentence_embedding])[0][0]
                if similarity < min_similarity:
                    if len(' '.join(current_chunk[:-1])) >= min_length:
                        chunks.append(' '.join(current_chunk[:-1]))
                    current_chunk = [sentence]
                    current_embedding = sentence_embedding
            else:
                current_embedding = sentence_embedding
    
    # Add the last chunk if it meets the min_length constraint
    if current_chunk and len(' '.join(current_chunk)) >= min_length:
        chunks.append(' '.join(current_chunk))
    
    return chunks

## Data loading example

In [3]:
# Read train.csv and test.csv
train_df = pd.read_csv("/kaggle/input/juridia-hackhaton-fine-tuning-llm-v/train.csv")
test_df = pd.read_csv("/kaggle/input/juridia-hackhaton-fine-tuning-llm-v/test.csv")


print("Train Data:")
display(train_df.head())
print("Test Data:")
display(test_df.head())

Train Data:


,file_name,long_title,date,doc_type,Id
0,8293.html,Décret n° 2-04-534 du 16 kaada 1425 (29 décemb...,2004-12-29,Décret,0d5401c3-6d59-4a9f-bbe8-4313b599e94b
1,9003.html,Décret n° 2-72-513 du 3 rebia I 1393 (7 avril ...,1973-04-07,Décret,fea3e4a6-7535-425f-bdf2-b792f6052cd5
2,11506.html,Dahir n° 1-03-300 du 2 rabii I 1425 (22 avril ...,2004-04-22,Dahir,b67a1c82-9fc7-4a4b-8c67-10f45f5b9dd2
3,22837.html,Arrêté du ministre de l’économie et des financ...,2019-05-30,Arrêté,7bace715-97c7-47d5-a321-d17e1bee26a8
4,7066.html,Dahir n° 1-17-15 du 28 ramadan 1438 (23 juin 2...,2017-06-23,Dahir,5590ac6b-aa88-4b0d-af35-633fbbeb0bbf


Test Data:


,question
0,Quels sont les critères d'éligibilité à la ret...
1,Quelle est la durée maximale d'un contrat de t...
2,Quelle est la définition légale d'une donation ?
3,Comment fonctionne la médiation dans un litige...
4,Quelles sont les étapes d'une procédure de div...


In [7]:
import os
import requests
from IPython.display import display, HTML

def read_html_file_from_azure(file_name):
    # Azure Blob Storage details
    base_url = "https://juridiahackhaton.blob.core.windows.net/fine-tuning"
    sas_token = "?sp=r&st=2024-12-18T14:00:31Z&se=2024-12-23T22:00:31Z&skoid=84f95543-1f4f-468b-88b0-7ba29b72fff8&sktid=75319ac2-1960-4be6-b7bf-49f186bd8d73&skt=2024-12-18T14:00:31Z&ske=2024-12-23T22:00:31Z&sks=b&skv=2022-11-02&spr=https&sv=2022-11-02&sr=c&sig=G1chY5U9%2BocqupQtO%2FPZ0i4V4hxWUUD%2B09tYqZUAw2M%3D"
    file_name = "documents/" + file_name
    # Construct the complete blob URL with SAS token
    blob_url = f"{base_url}/{file_name}{sas_token}"

    # Fetch the HTML file
    response = requests.get(blob_url)

    if response.status_code == 200:
        # Display the HTML file content
        html_content = response.text
        return html_content
    else:
        print(f"Failed to fetch the HTML file: HTTP {response.status_code}")
        print(f"Error details: {response.text}")
        return False

# Example usage
html_content = read_html_file_from_azure("8293.html")

In [8]:
with open("juridia.html", "w") as file:
    file.write(html_content)

In [9]:
print("HTML File Content:")
display(HTML(html_content))

HTML File Content:


In [7]:
from bs4 import BeautifulSoup


def process_list(input_list):
    """
    Processes a list of strings, replacing '\xa0' with a space
    if the next element is not '\xa0', otherwise adding a newline.
    """
    result = ""
    for i, item in enumerate(input_list):
        if item == '\xa0':
            if i + 1 < len(input_list) and input_list[i + 1] != '\xa0':
                result += " "  # Add a space
        else:
            result += item
            if i + 1 < len(input_list) and input_list[i + 1] != '\xa0':
                result += "\n"  # Add a new line
    return result


def prepare_html_file(file_name):
    html_content=read_html_file_from_azure(file_name)
    soup = BeautifulSoup(html_content, 'html.parser')
    fr_tags = soup.find_all(attrs={"lang": "FR"})
    new_content=[content.text for content in fr_tags]
    return  process_list(new_content)

In [9]:
html_string=prepare_html_file("8293.html")
#print(html_string)

In [13]:
import pandas as pd

def preprocess_raw_text(df, output_path):
    # Load tabular data
    
    # Combine metadata with content if necessary
    json_data = []
    for _, row in df.iterrows():
        text = f"Title: {row['long_title']}\nDate: {row['date']}\nType: {row['doc_type']}\n\n{row['article_content']}"
        json_data.append({"text":text})

    # Save to a text file, one document per line
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, indent=4, ensure_ascii=False)



In [29]:
from tqdm import tqdm  # Import tqdm for progress bar
from concurrent.futures import ThreadPoolExecutor
from datasets import Dataset  

def preprocess_and_convert_to_dataset(df):
    """
    Preprocess the DataFrame and convert it to a datasets.Dataset instance.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns: long_title, date, doc_type, article_content, file_name.
    
    Returns:
        Dataset: A Hugging Face Dataset instance where each row is a sample.
    """
    texts = []  # List to store 'text' field values

    # Use tqdm to display a progress bar for the file processing
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(prepare_html_file, df["file_name"]), total=len(df), desc="Processing HTML files"))

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        html_content = results[idx]
        # Combine the metadata with the HTML content
        text = (
            f"Title: {row['long_title']}\n"
            f"Date: {row['date']}\n"
            f"Type: {row['doc_type']}\n\n"
            f"{html_content}"
        )
        texts.append({"text": text})  # Append the individual text entry

    # Convert the list into a Hugging Face Dataset
    dataset = Dataset.from_list(texts)
    return dataset


In [28]:
dataset_json=preprocess_and_convert_to_dataset(train_df)

Processing rows: 100%|██████████| 5000/5000 [00:01<00:00, 4519.80it/s]


In [30]:
dataset_json

Dataset({
    features: ['text'],
    num_rows: 5000
})

In [ ]:
chunks = text_chunking(text, max_chunk_size=100,min_length=50,model=model,tokenizer=tokenizer,min_similarity=0.85)


In [ ]:
from datasets import Dataset

max_chunk_size = 100
min_length = 50
min_similarity = 0.85

all_chunks = []

for i in range(len(data)):
    text = data[i]['text']
    chunks = text_chunking(
        text, 
        max_chunk_size=max_chunk_size, 
        min_length=min_length, 
        model=model, 
        tokenizer=tokenizer, 
        min_similarity=min_similarity
    )
    all_chunks.extend(chunks)  


new_dataset = Dataset.from_dict({'text': all_chunks})

# Preview the new Dataset
print(new_dataset)


In [31]:
from huggingface_hub import login
login(token="hf_api_key")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [35]:
dataset_json.push_to_hub("legal_dataset")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mouadenna/legal_dataset/commit/8a84e429dcf8ae700b379286930d25885455aabe', commit_message='Upload dataset', commit_description='', oid='8a84e429dcf8ae700b379286930d25885455aabe', pr_url=None, pr_revision=None, pr_num=None)